
# Import Python Packages


In [1]:
from tensorflow.python.keras.backend import clear_session
from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.python.keras.utils.layer_utils import print_summary
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import load_model
import numpy as np
from CLRNet_convlstm import CLRNet
from Utility_functions import create_sequence
from datetime import datetime as dt
from DFVDSequence import DFVDSequence
import matplotlib.pyplot as plt
print("yolo")
from sklearn.metrics import classification_report

# import shutil

# from src.xception_convlstm import Xception
# print(cv2.__version__)
# import random
# random.seed(32)

dataset_dir='DeepFakeDatasetReal'
# from src.cl_basic import cl_basic
import ipykernel
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import backend as K
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# Choose GPU NUMBERS [0, 1, 2, 3]
os.environ["CUDA_VISIBLE_DEVICES"]="0"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)
sess.run(tf.global_variables_initializer())

# Load Validation Dataset for rechecking accuracy

In [2]:
frames_per_video_per_batch=5
frames_per_video=80
image_size=240
test_video_per_batch=16
data_augmentation=False
X_FaceSwap,y_FaceSwap,class_weights_FaceSwap=create_sequence('datasets/FaceSwap/val/',frames_per_video_per_batch,frames_per_video)
FaceSwap_it=DFVDSequence(X_FaceSwap,y_FaceSwap,test_video_per_batch,frames_per_video_per_batch,image_size,data_augmentation)

In [3]:
fig=plt.figure(figsize=(10, 100))
columns = 5
rows = 40

print("FaceSwap")
x=1
temp=FaceSwap_it.__getitem__(1)
fig.add_subplot(rows, columns, x)
plt.imshow(np.array(temp[0][0,0]))
fig.add_subplot(rows, columns, x+1)
plt.imshow(np.array(temp[0][0,1]))
fig.add_subplot(rows, columns, x+2)
plt.imshow(np.array(temp[0][0,2]))
fig.add_subplot(rows, columns, x+3)
plt.imshow(np.array(temp[0][0,3]))
fig.add_subplot(rows, columns, x+4)
plt.imshow(np.array(temp[0][0,4]))
print("Min:",np.array(temp[0][0,0]).min(),"Max:",np.array(temp[0][0,0]).max())
fig.add_subplot(rows, columns, x+5)
plt.imshow(np.array(temp[0][3,0]))
fig.add_subplot(rows, columns, x+6)
plt.imshow(np.array(temp[0][3,1]))
fig.add_subplot(rows, columns, x+7)
plt.imshow(np.array(temp[0][3,2]))
fig.add_subplot(rows, columns, x+8)
plt.imshow(np.array(temp[0][3,3]))
fig.add_subplot(rows, columns, x+9)
plt.imshow(np.array(temp[0][3,4]))
print("Min:",np.array(temp[0][1,0]).min(),"Max:",np.array(temp[0][1,0]).max())
print(temp[0].shape,temp[1].shape,FaceSwap_it.dataset_size,FaceSwap_it.frame_counter,FaceSwap_it.__len__(),temp[1][0],temp[1][1])
plt.title("Training Set Examples")        
plt.show()
FaceSwap_it.on_epoch_end()

# Load Model

In [4]:
model = load_model('models/FaceSwap/CLRNet50(bk)/19-12-17-10_03/CLRNet50(bk)_19-12-17-10_03-130-0.99.hdf5')
model.summary()
adam_fine = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='binary_crossentropy', optimizer=adam_fine, metrics=['accuracy'])

In [5]:
predictions = model.predict_generator(FaceSwap_it, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = FaceSwap_it.pre_classes

In [6]:
# class_labels = list(FaceSwap_it.class_indices.keys()) 
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [6]:

print(FaceSwap_it.pre_classes)
# print(len(predictions))
# print(len(y_FaceSwap))

# DeepFakes Test

### Load DeepFakes Test Dataset

In [7]:
frames_per_video_per_batch=5
frames_per_video=80
image_size=240
test_video_per_batch=16
data_augmentation=False
X_DeepFake,y_DeepFake,class_weights_DeepFake=create_sequence('datasets/Deepfakes/test/',frames_per_video_per_batch,frames_per_video)
DeepFake_it=DFVDSequence(X_DeepFake,y_DeepFake,test_video_per_batch,frames_per_video_per_batch,image_size,data_augmentation)

### Predict DeepFakes

In [8]:
predictions = model.predict_generator(DeepFake_it, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = DeepFake_it.pre_classes
# class_labels = list(DeepFake_it.class_indices.keys()) 
report = classification_report(true_classes, predicted_classes,  digits=4)
print(report)

# FaceSwap Test

### Load FaceSwap Test Dataset

In [9]:
frames_per_video_per_batch=5
frames_per_video=80
image_size=240
test_video_per_batch=16
data_augmentation=False

X_FaceSwap,y_FaceSwap,class_weights_FaceSwap=create_sequence('datasets/FaceSwap/test/',frames_per_video_per_batch,frames_per_video)
FaceSwap_it=DFVDSequence(X_FaceSwap,y_FaceSwap,test_video_per_batch,frames_per_video_per_batch,image_size,data_augmentation)


### Predict FaceSwap

In [10]:
predictions = model.predict_generator(FaceSwap_it, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = FaceSwap_it.pre_classes
# class_labels = list(FaceSwap_it.class_indices.keys()) 
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# Face2Face Test

### Load Face2Face Test Dataset

In [11]:
frames_per_video_per_batch=5
frames_per_video=80
image_size=240
test_video_per_batch=16
data_augmentation=False
X_Face2Face,y_Face2Face,class_weights_Face2Face=create_sequence('datasets/Face2Face/test/',frames_per_video_per_batch,frames_per_video)
Face2Face_it=DFVDSequence(X_Face2Face,y_Face2Face,test_video_per_batch,frames_per_video_per_batch,image_size,data_augmentation)

### Predict Face2Face

In [12]:
predictions = model.predict_generator(Face2Face_it, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = Face2Face_it.pre_classes
# class_labels = list(Face2Face_it.class_indices.keys()) 
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)